# Tutorial 8: Obtaining Realistic Cherenkov Events

While it is possible to create toy Cherenkov events with `PhotoelectronSource.get_cherenkov_shower`, there are no limitations on the parameters supplied. The resulting Cherenkov showers can consequently be very unrealistic. Additionally, there is no underlying relation of the parameters to the typical physical energy spectra. To reliably describe the on-sky performance of a camera simulated with sstcam-simulation, we must be able to simulate its response to realistic Cherenkov showers.

This tutorial demonstrates the `PhotoelectronReader` class, reading a photoelectron HDF5 file created from the photoelectrons saved in simtelarray files. The information included in each row of this HDF5 table is the arrival time and arrival pixel for each photoelectron, as well as the Cherenkov shower simulation parameters (e.g. energy)

In [1]:
from sstcam_simulation import Camera, PhotoelectronReader, EventAcquisition, PhotoelectronSource
from sstcam_simulation.camera.pulse import GaussianPulse
from sstcam_simulation.plotting import CameraImage
from sstcam_simulation.data import get_data
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
camera = Camera(
    continuous_readout_duration=128,
)

In [3]:
# Obtain test dataset
photoelectron_path = get_data("testing/photoelectron_test.h5")

In [4]:
# Obtain first Cherenkov event in file
with PhotoelectronReader(photoelectron_path, camera, start_time=40) as reader:
    for pe in reader:
        if len(pe) > 100:
            break

# Simulate some nsb
source = PhotoelectronSource(camera=camera)
nsb = source.get_nsb(10)
acquisition = EventAcquisition(camera=camera)
readout = acquisition.get_continuous_readout(pe+nsb)
samples = acquisition.get_sampled_waveform(readout)

Opening HDF5 file: /Users/Jason/Software/sstcam-simulation/sstcam_simulation/data/testing/photoelectron_test.h5
Reading entire DataFrame (geometry) from HDF5 file into memory
Reading entire DataFrame (data) from HDF5 file into memory


In [5]:
image = CameraImage.from_coordinates(camera.mapping.pixel)
image.add_colorbar("Total Number of Photoelectrons")
image.image = pe.get_charge_per_pixel(camera.mapping.n_pixels)

In [6]:
image = CameraImage.from_coordinates(camera.mapping.pixel)
image.add_colorbar("Waveform Amplitude")
image.set_limits_minmax(samples.min(), samples.max())
n_samples = samples.shape[1]
for isample in range(n_samples):
    image.image = samples[:, isample]
    plt.pause(0.001)

In [7]:
_ = plt.plot(samples.T)